In [ ]:
import ee
import geemap
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps
import sys
from ipywidgets import Button, DatePicker , Layout, VBox, HTML, FileUpload, Dropdown, Text, Textarea, widgets
from shapely.geometry import shape, Point, mapping
from ipyleaflet import WidgetControl, ImageOverlay, Marker, Popup, GeoData, basemaps, LayersControl
import ipywidgets as widgets
import uuid
import json
from geemap import geojson_to_ee
import geojson
from geojson import GeometryCollection, Point, LineString
import tkinter as tk
import turtle
from ipyfilechooser import FileChooser
import geopandas as gpd
import os
import tempfile
import functools
from datetime import datetime, timedelta

In [ ]:
conn = ps.connect(dbname="railway",
                    user="postgres",
                    password="fBEaF2Aab2gd1gEaB144f33Da6aCBa1d",
                    host="monorail.proxy.rlwy.net",
                    port="42210")

In [ ]:
import warnings

warnings.filterwarnings('ignore')

cur = conn.cursor()

In [ ]:
try:
    service_account = ' geositeconta@geosite-1653095517515.iam.gserviceaccount.com'
    credentials = ee.ServiceAccountCredentials(service_account, 'token.json')
    ee.Initialize(credentials)
    
    #print('Google Earth Engine has initialized succesfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialized!', e)
except:
    print('Unexpected error:', sys.exc_info()[0])
    raise

In [ ]:
Map = geemap.Map(center=[-24.960321, -53.485825], zoom=11)
Map.add_basemap('HYBRID')  # Add Google Satellite
#Map.setCenter(-53.485825, -24.960321, 10)
mc = Map.layer_control
Map.remove_control(mc)

In [ ]:
colorNdvi = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',  '74A901', '66A000', '529400',  '3E8601',  '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']
colorGari = ['006837', '1A9850', 'FFFFBF', 'FD8D3C', 'A50026']
colorEvi = ['FFFFFF', 'E5F5E0', '31A354', '006837']
colorNdwi = ['ece7f2', 'd0d1e6', 'a6bddb', '74a9cf', '3690c0', '0570b0', '045a8d', '023858']
colorGndvi = ['FF0000', 'FFA500', 'FFFF00', '00FF00', '0000FF', '800080']
colorNdre = ['008000', '00FF00', 'FFFF00', 'FF0000']
colorOsavi = ['8B0000', 'FFFFFF', '008000']
colorGsavi = ['004D00', '008000', 'D9D9C3', 'B3B300', '8B4513']
colorNavi = ['000080', '00FFFF', 'ADFF2F', '008000']
colorRvi = ['ff0000', '00FF00']
colorSavi = ['8B0000', 'FF0000', 'FFFF00', '00FF00', '008000']

In [ ]:
load_button = Button(description="Filtrar Data")

start_date = "2023-04-01"
end_date = "2023-04-30"


# Create the date picker widgets
start_date_picker = DatePicker(description='Data Inicial:')
end_date_picker = DatePicker(description='Data Final:')

# Initialize the images and other dictionaries
images = {}
feature_dict = {}
data_to_insert = []
data_to_insert_handle_click = []

collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filter(ee.Filter.lt('CLOUD_COVER_LAND', 10)).filter(ee.Filter.lt('CLOUD_COVER', 10))

# Data Inicial e Final padrões caso o usuário não escolha
yesterday = datetime.now() - timedelta(days=1)
start_date_default = yesterday - timedelta(days=365)  # A data de início padrão é um ano atrás
end_date_default = yesterday  # A data final padrão é ontem

start_date_default = ee.Date(start_date_default.strftime('%Y-%m-%d'))
end_date_default = ee.Date(end_date_default.strftime('%Y-%m-%d'))



    

#collection = collection.filterDate(start_date_default, end_date_default)

collection = collection.filterDate('2023-05-01', '2023-05-30')

filtered_collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA').filter(ee.Filter.lt('CLOUD_COVER_LAND', 10)).filter(ee.Filter.lt('CLOUD_COVER', 10))
test = ee.ImageCollection([])

global index_collection

def update_index_collection(start_date, end_date):
    global collection, index_collection, test, median_ndvi
    

    # Filtrar a collection com base nas datas selecionadas
    collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_TOA') \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER_LAND', 10)) \
        .filter(ee.Filter.lt('CLOUD_COVER', 10))

    # Check if the collection is empty
    if collection.size().getInfo() == 0:
        print("Nenhuma imagem disponível entre as datas selecionadas")
        return

    # Recalcula os índices com a coleção filtrada
    index_collection = collection.map(calcIndex)

    # Calculate median NDVI
    median_ndvi = index_collection.select('NDVI').median()

def on_button_click(b):
    global test

    start_date = start_date_picker.value
    end_date = end_date_picker.value

    if not start_date or not end_date:
        print("Selecione datas válidas")
        return

    # Format start_date and end_date correctly
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    test = collection

    # Atualiza o mapa
    selected_index = index_dropdown.value
    
def calcIndex(img):
    global test  # Access the global variable directly

    nir = img.select('B5')
    red = img.select('B4')
    green = img.select('B3')
    blue = img.select('B2')
    
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI');
    
    gari = green.subtract(red).divide(green.add(red.subtract(blue))).multiply(green.subtract(nir)).divide(green.add(nir)).rename('GARI');
    
    evi = ((nir.subtract(red)).divide(nir.add(6)).multiply(red.subtract(7.5).multiply(blue.add(1))).multiply(2.5)).rename('EVI');

    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI');

    gndvi = nir.subtract(green).divide(nir.add(green)).rename('GNDVI');

    ndre = nir.subtract(red).divide(nir.add(red)).rename('NDRE');

    osavi = nir.subtract(red).divide(nir.add(red).add(0.16)).rename('OSAVI');

    gsavi = (nir.subtract(green).divide(nir.add(green).add(0.5)).divide(1.5)).rename('GSAVI');

    navi = (red.divide(nir)).divide(1).rename('NAVI');

    rvi = nir.divide(red).rename('RVI');

    savi = (nir.subtract(red).divide(nir.add(red).add(0.5))).multiply(1.5).rename('SAVI');

    return img.addBands(ndvi).addBands(gari).addBands(evi).addBands(ndwi).addBands(gndvi).addBands(ndre).addBands(osavi).addBands(gsavi).addBands(navi).addBands(rvi).addBands(savi)


#index_collection = test.map(calcIndex)
index_collection = collection.map(calcIndex)

#index_collection = collection.map(lambda img: calcIndex(img, collection))
index_image = ee.ImageCollection(index_collection).toBands()


update_index_collection(start_date_default, end_date_default)

# Connect the function to the button click event
load_button.on_click(on_button_click)
    
# Create the WidgetControl
buttonDateStart = WidgetControl(widget=start_date_picker, position='topright')
buttonDateEnd = WidgetControl(widget=end_date_picker, position='topright')
buttonLoad = WidgetControl(widget=load_button, position='topright')

# Add the WidgetControl to the map
Map.add_control(buttonDateStart)
Map.add_control(buttonDateEnd)
Map.add_control(buttonLoad)

ndviParams = {'min': -1, 'max': 1, 'palette': colorNdvi}

gariParams = {'min': -1, 'max': 1, 'palette': colorGari}

eviParams = {'min': -1, 'max': 1, 'palette': colorEvi}

ndwiParams = {'min': -1, 'max': 1, 'palette': colorNdwi}

gndviParams = {'min': -1, 'max': 1, 'palette': colorGndvi}

ndreParams = {'min': -1, 'max': 1, 'palette': colorNdre}

osaviParams = {'min': -1, 'max': 1, 'palette': colorOsavi}

gsaviParams = {'min': -1, 'max': 1, 'palette': colorGsavi}

naviParams = {'min': -1, 'max': 1, 'palette': colorNavi}

rviParams = {'min': -1, 'max': 1, 'palette': colorRvi}

saviParams = {'min': -1, 'max': 1, 'palette': colorSavi}


median_ndvi = index_collection.select('NDVI').median()
median_gari = index_collection.select('GARI').median()
median_evi = index_collection.select('EVI').median()
median_ndwi = index_collection.select('NDWI').median()
median_gndvi = index_collection.select('GNDVI').median()
median_ndre = index_collection.select('NDRE').median()
median_osavi = index_collection.select('OSAVI').median()
median_gsavi = index_collection.select('GSAVI').median()
median_navi = index_collection.select('NAVI').median()
median_rvi = index_collection.select('RVI').median()
median_savi = index_collection.select('SAVI').median()


In [ ]:
index_options = ['EVI', 'GARI', 'GNDVI', 'GSAVI', 'NAVI', 'NDRE', 'NDVI', 'NDWI', 
                 'OSAVI', 'RVI', 'SAVI']

# Create a dropdown widget for the index options
index_dropdown = widgets.Dropdown(options=index_options)

index_dict = {
    'EVI': (median_evi, eviParams),
    'GARI': (median_gari, gariParams),
    'GNDVI': (median_gndvi, gndviParams),
    'GSAVI': (median_gsavi, gsaviParams),
    'NAVI': (median_navi, naviParams),
    'NDRE': (median_ndre, ndreParams),
    'NDVI': (median_ndvi, ndviParams),
    'NDWI': (median_ndwi, ndwiParams),
    'OSAVI': (median_osavi, osaviParams),
    'RVI': (median_rvi, rviParams),
    'SAVI': (median_savi, saviParams),          
}

# Create the WidgetControl
control = WidgetControl(widget=index_dropdown, position='topright')

#Map.add_control(control)

In [ ]:
from ipywidgets import HTML, Dropdown, Text, Textarea, widgets
from shapely.geometry import shape, Point, mapping

feat_list = []

# Create the draw control
dc = Map.draw_control
dc.rectangle = {"shapeOptions": {"fillColor": "transparent", "color": "black"}}
dc.polygon = {"shapeOptions": {"fillColor": "transparent", "color": "black"}}
dc.circle = {"shapeOptions": {"fillColor": "transparent", "color": "black"}}
dc.edit = True
dc.remove = False #trash button 

# Create a dictionary to store the features and images for each index type
images = {}

data_to_insert = []
data_to_insert_handle_click = []

data_to_insert_dict = {}

storeTxtInput = []
som = []
shape_dict = {}

feature_dict = {}

store_history = []

geojsonStr = ""
selected_index = ""
geojsonStr = ""
area_ha = 0

popup_dict = {}

geojsonStrArray = []
selected_indexArray = []
geojsonStrArray = []
area_haArray = []

test = []
popup_text_values = []
global popup_text_values, test
input_text_widgets = [] #// {} = dictionary, [] = list

# This will store the polygons
drawn_features = []
indice_labels = {}


# Handle draw events
def handle_draw(self, action, geo_json):
    global images, feature_dict, data_to_insert, data_to_insert_handle_click
    global geojsonStr, selected_index, geojsonStr, area_ha
    global geojsonStrArray, selected_indexArray, geojsonStrArray, area_haArray
    global popup_dict

    selected_index = index_dropdown.value    
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value

    start_date = "2023-09-01"
    end_date = "2023-09-30"
    
    # Get the index image and parameters
    index_image, index_params = index_dict[selected_index]

    geom = geojson_to_ee(geo_json, False)

    feature = ee.Feature(geom)
    feature = feature.set("index", str(selected_index))
    feature = feature.set("selected", False)
    feature = feature.set("id", str(uuid.uuid4()))
    
    if selected_index not in images:
        images[selected_index] = ([], [])
        
    geojson_dict = geom.getInfo()
    geojsonStr = geojson.dumps(geojson_dict)
    
    # Convert the geom variable to a GEOMETRY type
    geom_hash = hash(geojsonStr)
    geom_geometry = "GEOMETRY" + str(geom_hash)
    
    # Calculate the area of the drawn geometry and print it in hectares
    area_m2 = geom.area().getInfo()
    area_ha = int(area_m2 / 10000)
    
    geojsonStrArray = geojsonStr
    selected_indexArray = selected_index
    geojsonStrArray = geojsonStr
    area_haArray = area_ha
  
    data_to_insert.append((geojsonStr, str(selected_index), geojsonStr, area_ha))
    #print('handle_draw:',data_to_insert)
    data_to_insert_handle_click = data_to_insert
    #se desta forma acima nao resolver, basta eu criar outras variaveis que possam
    #armazenar esses dados em uma lista ou array para cada valor e depois usar o 
    #data_to_insert.append la no saveShape

    from IPython.display import display
   
    description = widgets.Label(value='Talhao:',layout=widgets.Layout(align_items='flex-start', align_self='initial'))
    input_text = widgets.Text(value='', placeholder='Insira seu texto aqui',layout=widgets.Layout(width='auto', height='auto'))
    input_text.drawing_id = None 
    input_text_widgets.append(input_text)


    box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='initial')
    widget_box = widgets.VBox([description, input_text], layout=box_layout)

    #display(widget_box)

    # create widgets for the strings
    indice_label = widgets.Label(value=f"Indice: {selected_index}")
    area_label = widgets.Label(value=f"Area: {area_ha} ha")
    
    popupBox = widgets.VBox([indice_label, widget_box, area_label],
                            layout=widgets.Layout(display='flex', align_items='flex-start', align_self='initial'))
    
    popup2 = Popup(
        location=(geom.centroid().getInfo()['coordinates'][1], geom.centroid().getInfo()['coordinates'][0]),
        child=popupBox,
        close_button=True,
        auto_close=False,
        close_on_escape_key=True
    )

    Map.add_layer(popup2)
    marker = Marker(location=(geom.centroid().getInfo()['coordinates'][1], geom.centroid().getInfo()['coordinates'][0]))
    marker.popup = popup2
    Map.add_layer(marker)
    
    if selected_index in images:
        feat_list, img_list = images[selected_index]
        feat_list.append(feature)
        img_list.append((index_image, index_params))
    else:
        images[selected_index] = ([feature], [(index_image, index_params)])
    
    if selected_index in feature_dict:
        feature_dict[selected_index].append(feature)
    else:
        feature_dict[selected_index] = [feature]

        

    # Add only the new images to the map
    feat_list, img_list = images[selected_index]
    collection = ee.FeatureCollection(feat_list).set({
        'system:time_start': ee.Date(start_date).millis(),
        'system:time_end': ee.Date(end_date).millis()
    })
    #clip_image = index_image.clipToCollection(collection)

        
    clip_image = index_image.clipToCollection(collection).set({
        'system:time_start': ee.Date(start_date).millis(),
        'system:time_end': ee.Date(end_date).millis()
    })

    #print("CLIP_IMAGE", clip_image)
    
   # print("Index image Info:", index_image.getInfo())

    feature_id = str(uuid.uuid4())
    popup_dict[feature_id] = popup2

    drawing_id = str(uuid.uuid4())
    feature = feature.set("id", drawing_id)
    input_text.drawing_id = drawing_id
    #input_text_widgets[drawing_id] = input_text

    indice_labels[feature_id] = indice_label
    feature = feature.set("id", feature_id)
    geo_json['properties']['id'] = feature_id

    # Add the feature to the images dictionary using its ID
    drawn_features.append(geo_json)    
    
     # Add the feature to the images dictionary using its ID
    layer_name = f"{feature_id}_{selected_index}"
    images[feature_id] = (feature, index_image, index_params, layer_name)
    geojsonStr = json.dumps(geo_json)
    data_to_insert_dict[feature_id] = (geojsonStr, str(selected_index), geojsonStr, area_ha)
    
    # When the dropdown value changes, fetch and display the image
    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            # Get the selected URL
            selected_url = change['new']

            # Get the corresponding id
            selected_id = list(thumbnails.keys())[list(thumbnails.values()).index(selected_url)]

            # Fetch the image for this id
            selected_image = ee.Image(geom_collection.filterMetadata('id', 'equals', selected_id).first())

            # Add the image to the map
            Map.addLayer(selected_image, {'bands': 'B4,B3,B2', 'min': 0, 'max': 0.3}, 'Selected Image')
 

    Map.add_ee_layer(clip_image, index_params, layer_name)
    
    out_img = os.path.expanduser("~/Downloads/landsat.png")
    clipped = clip_image.clip(geom)
    geemap.get_image_thumbnail(clipped, out_img, index_params, dimensions=300, format='png')

    geemap.show_image(out_img)
    
    # Refresh the map
    layers = Map.layers
    base_layer = layers[0]
    Map.clear_layers()
    Map.add_layer(base_layer)
    for layer in layers[1:]:
        Map.add_layer(layer)

    #ORIGINAL Map.add_ee_layer(clip_image, index_params, selected_index)

    # Define a function to update the storeTxtInput list with the new value
    unique_values = []

    def update_store(button):
        global popup_text_values, test
        
        if input_text.value not in popup_text_values:
            popup_text_values.append(input_text.value)

        testando = []
        for i, txt_input in enumerate(test):
            testando.append(txt_input)
            if input_text.value not in unique_values:
                unique_values.append(input_text.value)

        test = [txt_input for txt_input in testando if txt_input != popup_text_values[-1]]
        return test

    input_text.observe(update_store, names='value')

dc.on_draw(handle_draw)

def handle_click(**kwargs):
    global data_to_insert, geojsonStr, area_ha, collection
    if kwargs.get('type') == 'click':
        #print('CLICKED')
        lonlat = kwargs.get('coordinates')
        point = Point(lonlat[1], lonlat[0])  # Note the reversed order

        for i, feature in enumerate(drawn_features):
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                #print(f'Clicked inside polygon {feature}')

                # Get the feature's ID
                feature_id = feature['properties']['id']

                # Try to get the old layer and hide it
                try:
                    old_layer_name = images[feature_id][3]
                    old_layer = Map.find_layer(old_layer_name)
                    if old_layer is not None: 
                        old_layer.visible = False
                except KeyError:
                    pass

                # Get the new index from the dropdown
                new_selected_index = index_dropdown.value
                index_image, index_params = index_dict[new_selected_index]

                # Convert the feature to an Earth Engine object
                geom = geojson_to_ee(feature, False)

                # Create a feature collection from the geometry
                collection = ee.FeatureCollection([ee.Feature(geom)])

                # Clip the index image to the feature collection
                clip_image = index_image.clipToCollection(collection)

                # Add the new layer to the map with a new name
                new_layer_name = f"{feature_id}_{new_selected_index}"
                    
                # Remove the city layer, add the NDVI layer, and then re-add the city layer
                city_layer = Map.find_layer("City")
                if city_layer is not None:
                    Map.remove_layer(city_layer)
                Map.add_ee_layer(clip_image, index_params, new_layer_name)
                Map.add_layer(city_layer)
                
                out_img = os.path.expanduser("~/Downloads/landsat.png")
                clipped = clip_image.clip(geom)
                geemap.get_image_thumbnail(clipped, out_img, index_params, dimensions=300, format='png')

                geemap.show_image(out_img)

                # Update the images dictionary with the new layer
                images[feature_id] = (feature, index_image, index_params, new_layer_name)

                # Update the label with the new index
                indice_labels[feature_id].value = f"Indice: {new_selected_index}"
                geo_dic = geom.getInfo()
                geojsonStr = geojson.dumps(geo_dic)
                area_m2 = geom.area(1).getInfo()
                area_ha = int(area_m2 / 10000)

                # Convert the Earth Engine object back to the original feature format
                feature_dict = {
                    'type': 'Feature',
                    'geometry': geo_dic,
                    'properties': {
                        'id': feature_id,
                        'indice': new_selected_index
                    }
                }
                descricaoStr = json.dumps(feature_dict)
                data_to_insert_dict[feature_id] = (descricaoStr, new_selected_index, geojsonStr, area_ha)

                data_to_insert = []
                #print('ATT:',data_to_insert)
                data_to_insert.append((geojsonStr, str(new_selected_index), geojsonStr, area_ha))

                feature['properties']['indice'] = new_selected_index
                drawn_features[i] = feature
                
Map.on_interaction(handle_click)

def delLayer():
     global images, data_to_insert
     images = {}
     data_to_insert = []
     Map.remove_drawn_features()
     Map.remove_labels()
     Map.clear_layers()
     Map.add_basemap('HYBRID')
     Map.setCenter(-48.86154371799718, -21.53706897593839, 6)
    
delButton = Button(description="", icon="fa-solid fa-trash", layout=Layout(width='30px'))
delButton.on_click(lambda button: delLayer())

layerDel = WidgetControl(widget=delButton, position='topleft')
Map.add_control(layerDel) 

In [ ]:
def saveShape(s):
    global data_to_insert
    for i, data in enumerate(data_to_insert):
        ok = input_text_widgets[i].value
        input_text = input_text_widgets[i]
        talhao = input_text.value
        drawing_id = input_text.drawing_id
        data_maps_Insert = (*data, talhao, drawing_id)
        data_maps_Update = (*data, talhao)
      
        cur.execute("SELECT 1 FROM geomapping WHERE id = %s", (drawing_id,))
        exists = cur.fetchone() is not None
        if exists:
            cur.execute("UPDATE geomapping SET descricao = %s, indice = %s, geom = %s, hectare = %s, talhao = %s WHERE id = %s", (data_maps_Update + (drawing_id,)))
        else:
            cur.execute("INSERT INTO geomapping (descricao, indice, geom, hectare, talhao, id) VALUES (%s,%s,%s,%s,%s,%s)", data_maps_Insert)
    conn.commit()


load_data_button = Button(description="Salvar indices")
load_data_button.on_click(saveShape)

buttonSave = WidgetControl(widget=load_data_button, position='topright')

Map.add_control(buttonSave)


def load_data_from_db(b):
    global data_to_insert
    global input_text_widgets

    cur.execute("SELECT id, descricao, indice, geom, hectare, talhao FROM geomapping")
    rows = cur.fetchall()
    
    feature_dict = {}

    for row in rows:
        drawing_id, geojson_str, index_selected, geom, hectare, talhao = row
        geojson_data = json.loads(geojson_str)
        
        if geojson_data['type'] == 'Polygon':
            geom = ee.Geometry.Polygon(geojson_data['coordinates'])
        elif geojson_data['type'] == 'MultiPolygon':
            geom = ee.Geometry.MultiPolygon(geojson_data['coordinates'])
        else:
            raise ValueError(f"Invalid geometry type: {geojson_data['type']}")

        feature = {
            "type": "Feature",
            "geometry": geom.getInfo(),
            "properties": {
                'id': drawing_id,
                'indice': index_selected,
            }
        }

        # Append the feature to the drawn_features list
        drawn_features.append(feature)

        if index_selected in feature_dict:
            feature_dict[index_selected].append(feature)
        else:
            feature_dict[index_selected] = [feature]

    # Add the features to the map for each index type
    for index_selected, features in feature_dict.items():
        feature_collection = ee.FeatureCollection(features)
        index_image, index_params = index_dict[index_selected]
        clipped_image = index_image.clipToCollection(feature_collection)

        # Add the new layer to the map with a new name
        new_layer_name = f"{index_selected}"
        Map.addLayer(clipped_image, index_params, new_layer_name)

        # Update the images dictionary with the new layer
        for feature in features:
            images[feature['properties']['id']] = (feature, index_image, index_params, new_layer_name)

    data_to_insert_temp = []
    input_text_widgets_temp = []

    
    def style_callback(feature):
        return {
            'fillColor': 'transparent',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0
        }

    # Add the features to the map for each index type
    for index_selected, features in feature_dict.items():
        feature_collection = ee.FeatureCollection(features)
        index_image, index_params = index_dict[index_selected]
        clipped_image = index_image.clipToCollection(feature_collection)

        # Add the new layer to the map with a new name
        new_layer_name = f"{index_selected}"
        Map.addLayer(clipped_image, index_params, new_layer_name)

        # Create a GeoDataFrame from the features
        gdf = gpd.GeoDataFrame.from_features(features)

        # Create a GeoData layer and add it to the map
        geo_data = GeoData(geo_dataframe=gdf,
                           style_callback=style_callback,
                           hover_style={'fillColor': 'transparent', 'fillOpacity': 0},
                           name="City")
        Map.add_layer(geo_data)
        
    # Create a popup for each row with the corresponding talhao and hectare values
    for row in rows:
        drawing_id, geojson_str, index_selected, geom, hectare, talhao = row
        geojson_data = json.loads(geojson_str)
        geom = ee.Geometry.Polygon(geojson_data['coordinates'])

        description = widgets.Label(value='Talhao:',layout=widgets.Layout(align_items='flex-start', align_self='initial'))
        input_text = widgets.Text(value=talhao, placeholder='Insira seu texto aqui',layout=widgets.Layout(width='auto', height='auto'))
        input_text.drawing_id = drawing_id
        input_text_widgets_temp.append(input_text)

        indice_label = widgets.Label(value=f"Indice: {index_selected}")
        indice_labels[drawing_id] = indice_label  # Update indice_labels dictionary

        box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='initial')
        widget_box = widgets.VBox([description, input_text], layout=box_layout)
        hectare_label = widgets.Label(value=f"Hectare: {hectare}")
        popupBox = widgets.VBox([indice_label, widget_box, hectare_label], layout=widgets.Layout(display='flex', align_items='flex-start', align_self='initial'))

        popup = Popup(
            location=(geom.centroid().getInfo()['coordinates'][1], geom.centroid().getInfo()['coordinates'][0]),
            child=popupBox,
            close_button=True,
            auto_close=False,
            close_on_escape_key=True
        )

        marker = Marker(location=(geom.centroid().getInfo()['coordinates'][1], geom.centroid().getInfo()['coordinates'][0]))
        marker.popup = popup
        Map.add_layer(marker)
        
        # Prepare data for the data_to_insert list
        data_to_insert_temp.append((geojson_str, index_selected, geojson_str, hectare))
        
        # Update the data_to_insert and input_text_widgets lists
        data_to_insert = data_to_insert_temp
        input_text_widgets = input_text_widgets_temp
        
load_data_button = Button(description="Carregar Indices")
load_data_button.on_click(load_data_from_db)

buttonLoad = WidgetControl(widget = load_data_button, position = 'topright')

Map.add_control(buttonLoad)

In [ ]:
from fastkml import kml
from shapely.geometry import shape, mapping, Polygon
import geopandas as gpd
import zipfile
import json
import xml.etree.ElementTree as ET
import re
from pykml import parser


def geometry_to_geojson(geometry):
    if geometry is None:
        return None

    if geometry.type == 'Polygon':
        coordinates = list(geometry.exterior.coords)
    elif geometry.type == 'MultiPolygon':
        coordinates = [list(poly.exterior.coords) for poly in geometry]
    else:
        raise ValueError(f"Unsupported geometry type: {geometry.type}")

    return {
        "type": "Polygon",
        "coordinates": coordinates
    }


def extract_polygon_coordinates(polygon_element):
    coordinates = []
    if polygon_element is not None:
        coords = polygon_element.find(".//{http://www.opengis.net/kml/2.2}coordinates")
        if coords is not None and coords.text:
            coordinates_str = coords.text.strip()
            coordinates_list = coordinates_str.split()
            for coord_str in coordinates_list:
                parts = coord_str.split(',')
                if len(parts) == 3:
                    lon, lat, _ = parts
                    coordinates.append((float(lon), float(lat)))
    return coordinates

def extract_features(placemark):
    data_dict = {'geometry': [], 'description': [], 'name': []}
    
    if placemark is not None:
        # Suporte a outras geometrias pode ser adicionado aqui.
        polygon_element = placemark.find(".//{http://www.opengis.net/kml/2.2}Polygon")
        coordinates = extract_polygon_coordinates(polygon_element)
        
        if coordinates:
            data_dict['geometry'].append({'type': 'Polygon', 'coordinates': [coordinates]})
        else:
            # Logar ou processar outras geometrias aqui.
            pass
        
        description_elem = placemark.find(".//{http://www.opengis.net/kml/2.2}description")
        name_elem = placemark.find(".//{http://www.opengis.net/kml/2.2}name")
        data_dict['description'].append(description_elem.text if description_elem is not None else "")
        data_dict['name'].append(name_elem.text if name_elem is not None else "")
    
    return data_dict

import fastkml.kml

def extract_features_second_structure(feature):
    data_dict = {'geometry': [], 'description': [], 'name': []}

    if isinstance(feature, fastkml.kml.Placemark):
        if hasattr(feature, 'geometry'):
            data_dict['geometry'].append(feature.geometry)
            data_dict['description'].append(feature.description)
            data_dict['name'].append(feature.name)
    else:
        for child_feature in feature.features():
            data_dict_part = extract_features_second_structure(child_feature)
            data_dict['geometry'].extend(data_dict_part['geometry'])
            data_dict['description'].extend(data_dict_part['description'])
            data_dict['name'].extend(data_dict_part['name'])

    return data_dict



def read_kml(file):
    try:
        if file.lower().endswith('.kmz'):
            with zipfile.ZipFile(file, 'r') as z:
                kml_file = [name for name in z.namelist() if name.lower().endswith('.kml')][0]
                doc = z.read(kml_file)
        else:
            with open(file, 'rb') as myfile:  # Note the 'rb' mode for reading as bytes
                doc = myfile.read()

        root = ET.fromstring(doc)
        pl = root.find(".//{http://www.opengis.net/kml/2.2}Placemark")
        
        features = [] 

        for placemark in root.findall(".//{http://www.opengis.net/kml/2.2}Placemark"):
                # Extrair features de cada Placemark e adicionar à lista de features.
            data_dict = extract_features(placemark)
          
            if data_dict['geometry']:
                feature = {
                    "type": "Feature",
                    "properties": {
                        "description": data_dict['description'][0],
                        "name": data_dict['name'][0]
                    },
                    "geometry": data_dict['geometry'][0]
                }
                features.append(feature)
            
        if features:
            geojson_data = {
                "type": "FeatureCollection",
                "features": features
            }
            gdf = gpd.GeoDataFrame.from_features(geojson_data, crs="EPSG:4326")
            return gdf
        else:
            raise ValueError("No valid geometries found in the KML.")
    except Exception as e:
        print("Retorno de leitura do KML", e)
        raise
        #return None

def read_kml_with_both_structures(file):    
    try:
        if file.lower().endswith('.kmz'):
            with zipfile.ZipFile(file, 'r') as z:
                kml_file = [name for name in z.namelist() if name.lower().endswith('.kml')][0]
                doc = z.read(kml_file)
        else:
            with open(file, 'rb') as myfile:  # Note the 'rb' mode for reading as bytes
                doc = myfile.read()
    except Exception as e:
        print(f"Erro ao abrir ou ler o arquivo: {e}")
        return None
        
    try:
        # Tente ler usando o método existente
        return read_kml(file)
    except Exception as e:
        print(f"Não leu o primeiro método {e}. Tentando o segundo método...")
        
    # Se o primeiro método falhar, tente o segundo método
    try:
        k = kml.KML()
        k.from_string(doc)
        pl = list(k.features())[0]
        data_dict = extract_features_second_structure(pl)
        return gpd.GeoDataFrame(data_dict, crs="EPSG:4326")
    except Exception as e2:
        print(f"Erro com o segundo método: {e2}")
    
    print("Erro em read_kml_with_both_structures: Ambos os métodos falharam.")
    return None


    
os.environ['SHAPE_RESTORE_SHX'] = 'YES'

def load_shapefile(button):
    try:
        global geo_data
        if file_upload.value: # check if there is a file uploaded
            uploaded_filename = file_upload.value  # get the first (and only) item in the tuple
            for key, value in uploaded_filename.items():
                uploaded_filename = key
            if uploaded_filename.endswith('.kml'):
                gdf = read_kml_with_both_structures(uploaded_filename)
            else:
                gdf = gpd.read_file(uploaded_filename)
            gdf.crs = "EPSG:4326"  # replace 4326 with the appropriate EPSG code

            for _, row in gdf.iterrows():
                polygon = row.geometry
                feature_id = str(uuid.uuid4())  # generate a unique ID for the feature
                properties = row.drop('geometry').to_dict()  # get all other fields as properties
                properties['id'] = feature_id
                feature = {
                    'type': 'Feature',
                    'geometry': mapping(polygon), # convert polygon to GeoJSON
                    'properties': properties, 
                }

                drawn_features.append(feature)
                
                # Add feature to images dictionary
                selected_index = index_dropdown.value
                index_image, index_params = index_dict[selected_index]
                geom = geojson_to_ee(feature, False)
                collection = ee.FeatureCollection([ee.Feature(geom)])
                clip_image = index_image.clipToCollection(collection)

                layer_name = f"{feature_id}_{selected_index}"
                images[feature_id] = (feature, index_image, index_params, layer_name)

                # Create a label for this feature
                indice_labels[feature_id] = widgets.Label(value=f"Índice: {selected_index}")

            # Define the style callback function
            def style_callback(feature):
                return {
                    'fillColor': 'transparent',
                    'color': 'black',
                    'weight': 1,
                    'fillOpacity': 0
                }

            new_widget = Text(value='', placeholder='Digite seu texto', description='Talhão:', disabled=False)
            new_widget.drawing_id = feature_id  
            input_text_widgets.append(new_widget) 
            
            geo_data = GeoData(geo_dataframe=gdf,
                               style_callback=style_callback,
                               hover_style={'fillColor': 'transparent', 'fillOpacity': 0},
                               name="City")

            Map.add_layer(geo_data)
            
            out_img = os.path.expanduser("~/Downloads/landsat.png")
            clipped = clip_image.clip(geom)
            geemap.get_image_thumbnail(clipped, out_img, index_params, dimensions=300, format='png')

            translation_dict = {
                'description': 'descrição',
                'name': 'nome',
                # Add more translations as needed
            }
            
            geemap.show_image(out_img)
            # Add popups to each feature
            for feature in geo_data.data['features']:
                 properties = feature['properties']
                 translated_properties = {}
                 for k, v in properties.items():
                        translated_key = translation_dict.get(k, k)  # Use the translation or keep the original key
                        translated_properties[translated_key] = v
                 #properties_html = "<br>".join(f"{k}: {v}" for k, v in properties.items())
                 properties_html = "<br>".join(f"{k}: {v}" for k, v in translated_properties.items())

                 popup_content = VBox([new_widget, HTML(value=properties_html)])
                 geometry = shape(feature['geometry'])
                 if geometry.type == 'Polygon' or geometry.type == 'MultiPolygon':
                    center = [geometry.centroid.y, geometry.centroid.x]
                    Map.add_layer(Marker(location=center, draggable=False, title="Information", 
                                         popup=Popup(location=center, child=popup_content, close_button=True, auto_close=True, close_on_escape_key=True)))
        else:
            print("No file selected.")
    except AttributeError as ae:
        print("AttributeError caught:", ae)
    except ValueError as ve:
        print("ValueError caught:", ve)
    except ee.ee_exception.EEException as eeex:
        print("EEException caught:", eeex)
    except Exception as e:
        print("An unexpected error occurred:", e)


file_upload = widgets.FileUpload(
    accept='.shp,.kml',  
    multiple=False 
)

load_button = widgets.Button(description="Carregar Shp/Kml")

load_button.on_click(load_shapefile)

Map.add_control(WidgetControl(widget=file_upload, position='topright'))
Map.add_control(WidgetControl(widget=load_button, position='topright'))

info = widgets.HTML()

Map.add_control( WidgetControl(widget=info, position='bottomright'))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import widgets
from ipywidgets import Widget
from IPython.display import display

# Define the color palettes for different vegetation indices
color_palettes = {
    'EVI':   ["#FFFFFF", "#E5F5E0", "#31A354", "#006837"],
    'GARI':  ["#006837", "#1A9850", "#FFFFBF", "#FD8D3C", "#A50026"],
    'GNDVI': ['FF0000', 'FFA500', 'FFFF00', '00FF00', '0000FF', '800080'],
    'GSAVI': ['004D00', '008000', 'D9D9C3', 'B3B300', '8B4513'],
    'NAVI':  ['000080', '00FFFF', 'ADFF2F', '008000'],
    'NDRE':  ['008000', '00FF00', 'FFFF00', 'FF0000'],
    'NDVI':  ["#FFFFFF", "#CE7E45", "#DF923D", "#F1B555", "#FCD163", "#99B718",
             "#74A901", "#66A000", "#529400", "#3E8601", "#207401", "#056201",
             "#004C00", "#023B01", "#012E01", "#011D01", "#011301"],
    'NDWI':  ['ece7f2', 'd0d1e6', 'a6bddb', '74a9cf', '3690c0', '0570b0', '045a8d', '023858'],
    'OSAVI': ['8B0000', 'FFFFFF', '008000'],
    'RVI':   ['ff0000', '00FF00'],
    'SAVI':  ['8B0000', 'FF0000', 'FFFF00', '00FF00', '008000']
}

def hex_to_rgb(hex):
    return [int(hex[i:i+2], 16)/255 for i in (0, 2, 4)]

def create_colormap(colors):
    return mpl.colors.LinearSegmentedColormap.from_list("my_colormap", colors)

def create_colorbar(index_name):
    color_palette = color_palettes.get(index_name, [])  # Get the color palette for the selected index
    color_rgb = [hex_to_rgb(hex[1:]) for hex in color_palette]  # Omit the first character (#)

    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)

    cmap = create_colormap(color_rgb)
    norm = mpl.colors.Normalize(vmin=0, vmax=1)

    cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                    norm=norm,
                                    orientation='horizontal')
    cb1.set_label(index_name)
    plt.show()

index_options = ['EVI', 'GARI', 'GNDVI', 'GSAVI', 'NAVI', 'NDRE', 'NDVI', 'NDWI', 'OSAVI', 'RVI', 'SAVI']

index_dropdown = widgets.Dropdown(options=index_options)

def handle_dropdown_change(change):
    selected_index = change['new']
    create_colorbar(selected_index)

index_dropdown.observe(handle_dropdown_change, names='value')

control = WidgetControl(widget=index_dropdown, position='topright')

Map.add_control(control)

#display(control)




def create_colorbar(cmap, min_val, max_val, title):
    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)

    cmap = mpl.cm.get_cmap(cmap)
    norm = mpl.colors.Normalize(vmin=min_val, vmax=max_val)

    cb1 = mpl.colorbar.ColorbarBase(ax, cmap=cmap,
                                    norm=norm,
                                    orientation='horizontal')
    cb1.set_label(title)
    plt.show()

def hex_to_rgb(hex):
    return [int(hex[i:i+2], 16)/255 for i in (0, 2, 4)]

def create_colormap(colors):
    return mpl.colors.LinearSegmentedColormap.from_list("my_colormap", colors)

colorNdvi = ["#FFFFFF", "#CE7E45", "#DF923D", "#F1B555", "#FCD163", "#99B718",
                "#74A901", "#66A000", "#529400", "#3E8601", "#207401", "#056201",
                "#004C00", "#023B01", "#012E01", "#011D01", "#011301"]
colorGari = ['#006837', '#1A9850', '#FFFFBF', '#FD8D3C', '#A50026']
colorEvi = ['#FFFFFF', '#E5F5E0', '#31A354', '#006837']
colorNdwi = ['#ece7f2', '#d0d1e6', '#a6bddb', '#74a9cf', '#3690c0', '#0570b0', '#045a8d', '#023858']
colorGndvi = ['#FF0000', '#FFA500', '#FFFF00', '#00FF00', '#0000FF', '#800080']
colorNdre = ['#008000', '#00FF00', '#FFFF00', '#FF0000']
colorOsavi = ['#8B0000', '#FFFFFF', '#008000']
colorGsavi = ['#004D00', '#008000', '#D9D9C3', '#B3B300', '#8B4513']
colorNavi = ['#000080', '#00FFFF', '#ADFF2F', '#008000']
colorRvi = ['#ff0000', '#00FF00']
colorSavi = ['#8B0000', '#FF0000', '#FFFF00', '#00FF00', '#008000']

colorNdvi_rgb = [hex_to_rgb(hex[1:]) for hex in colorSavi]  # omit the first character (#)
cmap = create_colormap(colorNdvi_rgb)   

create_colorbar(cmap, min_val=-1, max_val=1, title='SAVI')



Map
